In [ ]:
%load_ext nb_black

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np


# Configuration and CONATANTS
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", 999)
DATADIR = "../data/moz/"
COLUMNS = ['Keyword', 'Min Monthly Volume', 'Max Monthly Volume', 
           'Specific Monthly Volume', 'Difficulty', 'Top Rank', 
           'Top Ranking URL', 'Site']

# Load all the MOZ keyword data CSVs into one giant datatframe.
dfs = []
for i, file in enumerate(Path(DATADIR).glob("moz_*.csv")):
    site = file.name.split("_")[1]
    df = pd.read_csv(file)
    df['Site'] = site
    df.columns = COLUMNS
    dfs.append(df)
df = pd.concat(dfs)

# replace non-finite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df["Specific Monthly Volume"] = df["Specific Monthly Volume"].astype(int)
df["Top Rank"] = df["Top Rank"].astype(int)

# Sort descending by volume & reset index
df.sort_values(by="Specific Monthly Volume", ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a pivot-table dataframe
df_pivot = pd.pivot_table(
        df,
        index=["Keyword"],
        columns=["Site"],
        values=["Top Rank"],
        aggfunc=["min"],
    ) #.reset_index(level=None)

# Set the index of the orinal df and the new df_pivot to Keyword
df.set_index("Keyword", inplace=True)
df_pivot = df_pivot.reset_index().set_index("Keyword")
df_pivot.columns = df_pivot.columns.droplevel().droplevel()

# Join the tables, sort desc by volume and drop dupes
df2 = df.join(df_pivot)
df2 = df2.sort_values(by="Specific Monthly Volume", ascending=False)
df2 = df2.reset_index().drop_duplicates(subset="Keyword", keep="first")

# Clean up the numbers in the Top Rank cells
df2.fillna("0", inplace=True)
for site in df2.columns[8:]:
    df2[site] = df2[site].astype(int)
for site in df2.columns[8:]:
    df2[site] = df2[site].replace(0, "")

# Get rid of unnecessary columns and turn Keyword into index
df2.drop("Min Monthly Volume", axis=1, inplace=True)
df2.drop("Max Monthly Volume", axis=1, inplace=True)
df2.drop("Top Ranking URL", axis=1, inplace=True)
df2.drop("Site", axis=1, inplace=True)
df2.set_index("Keyword", inplace=True)

# Save to Excel (without nuking the system)
df2 = df2[df2["Specific Monthly Volume"] >= 1000]
out_file = f"{DATADIR}/gap_analysis.xlsx"
writer = pd.ExcelWriter(out_file, engine='xlsxwriter')
df2.to_excel(writer)
writer.close()
print("Done")

# 